## 커맨드 디자인 패턴

커맨드 패턴은 객체가 특정 기능을 바로 수행하거나 나중에 트리거할 때 필요한 모든 정보를 캡슐화하는 행동 패턴이다. 캡슐화하는 정보는 다음과 같다.

* 메소드명
* 메소드를 소유하는 객체
* 메소드 인자

프린트 스풀러는 종이 크기, 가로/세로, 부수 등의 설정을 Command 객체에 저장한다. 인쇄를 요청하면 스풀러는 Command 객체의 execute() 함수를 호출하고 설정대로 인쇄한다.

## 커맨드 패턴 구성 요소

커맨드 패턴은 Command와 Receiver, Invoker, Client 클래스로 구성된다.

* Command 객체는 Receiver 객체에 대해 알고 있으며 Receiver 객체의 함수를 호출
* Receiver 함수의 인자는 Command에 저장돼 있다.
* Invoker는 명령을 수행한다.
* Client는 Command 객체를 생성하고 Receiver를 정한다.

커맨드 패턴의 목적

* 요청을 객체 속에 캡슐화
* 클라이언트의 다양한 요청을 매개변수화
* 유청을 큐에 저장
* 객체지향 콜백을 지원

커맨드 패턴의 적합한 상황

* 수행할 명령에 따라 객체를 변수화할 때
* 요청을 큐에 저장하고 각기 다른 시점에 수행해야 하는 경우
* 작은 단위의 연산을 기반으로 하는 상위 연산을 만들 때

In [3]:
class Wizard():
    def __init__(self, src, rootdir):
        self.choices = []
        self.rootdir = rootdir
        self.src = src
    
    def preferences(self, command):
        self.choices.append(command)
    
    def execute(self):
        for choice in self.choices:
            if list(choice.values())[0]:
                print("Copy binaries --", self.src, " to ", self.rootdir)
            else:
                print("No Operation")

# 클라이언트 코드
wizard = Wizard('python3.6.gzip', '/usr/bin/')

# 사용자가 원하는 파이썬 선택
wizard.preferences({'python': True})
wizard.preferences({'java': False})
wizard.preferences({'c': True})
wizard.execute()

Copy binaries -- python3.6.gzip  to  /usr/bin/
No Operation
Copy binaries -- python3.6.gzip  to  /usr/bin/


* Command: 연산을 수행할 인터페이스를 정의
* ConcreteCommand: Receiver 객체와 연산간 바인딩을 정의
* Client: ConcreteCommand 객체를 생성하고 Receiver를 설정
* Invoker: ConcreteCommand에 수행을 요청
* Receiver: 요청에 관련된 연산을 관리

클라이언트는 특정 연산을 요청하고 Invoker는 요청을 받아 캡슐화해 큐에 넣는다. ConcreteCommand 클래스는 이 요청을 책임지고 Receiver에 수행을 맡긴다.

In [5]:
from abc import ABCMeta, abstractmethod

class Command(metaclass = ABCMeta):
    def __init__(self, recv):
        self.recv = recv
        
    def execute(self):
        pass
    
class ConcreteCommand(Command):
    def __init__(self, recv):
        self.recv = recv
    
    def execute(self):
        self.recv.action()

class Receiver:
    def action(self):
        print("Receiver Action")

class Invoker:
    def command(self, cmd):
        self.cmd = cmd
    
    def execute(self):
        self.cmd.execute()

recv = Receiver()
cmd = ConcreteCommand(recv)
invoker = Invoker()
invoker.command(cmd)
invoker.execute()

Receiver Action


## 커맨드 패턴의 실제 활용 사례

